#### API Notes
https://docs.sleeper.app/#get-all-drafts-for-a-league

In [26]:
import pandas as pd
import numpy as np
import requests
import json

In [27]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Setting Initial Variables for use throughout program

In [28]:
#league_id = 384202455065427968   # Marco's League - 2019
#league_id = 384197315499626496   # Chunk's League - 2019
prev_league_id = 591137395731107840   # Marco's League - 2020
#prev_league_id = 588943105080733696   # Chunk's League - 2020
league_id = 722951233224155136   # Marco's League - 2021
#league_id = 712420605282045952   # Chunk's League - 2021


#new_yr_league_id = 588943105080733696   # Chunk's League

commissioner = 'Marco'
#commissioner = 'Chunk'

# Getting draft ID from sleeper, must be for previous year draft
#draft_number = 384202455065427969   # Marco's League - 2019
#draft_number = 468554237748441088   # Chunk's League - 2019
draft_number = 591137395731107841   # Marco's League - 2020
#draft_number = 588943105080733697   # Chunk's League - 2020


# new year of draft
season = 2021


num_teams = 10  # Marco's League
#num_teams = 12   # Chunk's League

### League Users

In [29]:
# api-endpoint
URL = "https://api.sleeper.app/v1/league/{}/users".format(league_id)

# sending get request and saving the response as response object 
r = requests.get(url = URL)

# extracting data in json format 
league = r.json() 

In [30]:
league_users = pd.DataFrame.from_dict(league, orient="columns")
league_users = league_users.loc[: , ['user_id','display_name','league_id']]
cols = ['user_id', 'league_id']
league_users[cols] = league_users[cols].applymap(np.int64)
#league_users

### League Rosters

In [31]:
# api-endpoint
URL = "https://api.sleeper.app/v1/league/{}/rosters".format(league_id)

# sending get request and saving the response as response object 
r = requests.get(url = URL) 

# extracting data in json format 
roster_data = r.json()

In [32]:
owners_data = pd.DataFrame.from_dict(roster_data, orient="columns")

# Keeping relevant columns, exploding out player list
owners_data = owners_data.loc[: , ['owner_id','roster_id','players','league_id']]
cols = ['owner_id','roster_id','league_id']
owners_data[cols] = owners_data[cols].applymap(np.int64)
#owners_data.head()

In [33]:
# Exploding out owner data to get individual players
rosters = owners_data.explode('players')
#rosters.head()

In [34]:
# Merging Owner and User Data
owners =  pd.merge(owners_data, league_users, how='left',
                  left_on = ['league_id','owner_id'], 
                  right_on = ['league_id','user_id'])

owners = owners.loc[: , ['display_name','owner_id','roster_id','league_id']]
#owners.head()

In [35]:
# Merging Rosters and Owner Data
rosters =  pd.merge(rosters, owners, how='left', on = ['league_id','roster_id', 'owner_id'])
#rosters = rosters.drop(['display_name_x', 'display_name_y'], axis=1)

#rosters.head()

## Getting Player Data

In [11]:
# api-endpoint 
URL = "https://api.sleeper.app/v1/players/nfl"

# sending get request and saving the response as response object 
r = requests.get(url = URL) 

# extracting data in json format 
plyr_data = r.json() 

In [12]:
all_player_df = pd.DataFrame.from_dict(plyr_data, orient="index")
all_player_df.head()

,injury_status,depth_chart_order,age,birth_country,depth_chart_position,pandascore_id,injury_start_date,metadata,injury_body_part,last_name,first_name,stats_id,search_first_name,birth_date,height,birth_state,status,espn_id,practice_participation,high_school,position,rotowire_id,sport,search_last_name,weight,sportradar_id,number,years_exp,search_full_name,fantasy_data_id,rotoworld_id,yahoo_id,hashtag,gsis_id,team,player_id,full_name,college,fantasy_positions,active,swish_id,news_updated,injury_notes,search_rank,birth_city,practice_description
2103,None,NaN,27.0,NaN,None,NaN,NaN,None,None,Booth,Cody,NaN,cody,1991-04-22,"6'5""",NaN,Inactive,17054.0,None,None,OT,9866.0,nfl,booth,285,4cd4976e-e230-4935-ad3f-c12876a41350,60.0,1.0,codybooth,16426.0,NaN,27841.0,#CodyBooth-NFL-FA-60,None,None,2103,Cody Booth,Temple,[OL],False,NaN,NaN,None,9999999.0,NaN,None
6250,NA,NaN,NaN,NaN,None,NaN,NaN,None,None,Bryant,Eurndraus,NaN,eurndraus,None,,NaN,Inactive,3916426.0,None,None,DT,NaN,nfl,bryant,,9ff46edb-988f-4c8a-ad56-0502808ca1a6,0.0,0.0,eurndrausbryant,21183.0,NaN,32538.0,#EurndrausBryant-NFL-FA-0,None,None,6250,Eurndraus Bryant,None,[DL],True,NaN,NaN,None,9999999.0,NaN,None
4116,None,NaN,26.0,NaN,None,NaN,NaN,None,None,Garcia,Antonio,NaN,antonio,1993-12-10,"6'6""",NaN,Inactive,2972820.0,None,None,OT,11794.0,nfl,garcia,293,a2b6e1f9-1f06-45e3-b6e9-79ec8490e5ef,71.0,3.0,antoniogarcia,18962.0,12247.0,30198.0,#AntonioGarcia-NFL-FA-71,00-0033922,None,4116,Antonio Garcia,Troy,[OL],True,NaN,1.560222e+12,None,9999999.0,NaN,None
6177,None,2.0,25.0,NaN,RDT,NaN,NaN,None,None,Saunders,Khalen,NaN,khalen,1996-08-09,"6'0""",NaN,Active,3121634.0,None,Parkway Central (MO),DT,13746.0,nfl,saunders,324,757c55e1-2f3a-41d2-a211-16bf577a1586,99.0,2.0,khalensaunders,20934.0,NaN,31916.0,#KhalenSaunders-NFL-KC-99,00-0035656,KC,6177,Khalen Saunders,Western Illinois,[DL],True,831799.0,1.603596e+12,None,1596.0,NaN,None
5870,None,1.0,24.0,NaN,QB,NaN,NaN,None,None,Jones,Daniel,879981.0,daniel,1997-05-27,"6'5""",NaN,Active,3917792.0,None,Charlotte Latin (NC),QB,13491.0,nfl,jones,220,0042266b-cb28-4012-bfd2-06650badad97,8.0,2.0,danieljones,20841.0,13963.0,31838.0,#DanielJones-NFL-NYG-8,00-0035710,NYG,5870,Daniel Jones,Duke,[QB],True,879981.0,1.630345e+12,None,113.0,NaN,None


In [13]:
all_player_df.columns

Index(['injury_status', 'depth_chart_order', 'age', 'birth_country',
       'depth_chart_position', 'pandascore_id', 'injury_start_date',
       'metadata', 'injury_body_part', 'last_name', 'first_name', 'stats_id',
       'search_first_name', 'birth_date', 'height', 'birth_state', 'status',
       'espn_id', 'practice_participation', 'high_school', 'position',
       'rotowire_id', 'sport', 'search_last_name', 'weight', 'sportradar_id',
       'number', 'years_exp', 'search_full_name', 'fantasy_data_id',
       'rotoworld_id', 'yahoo_id', 'hashtag', 'gsis_id', 'team', 'player_id',
       'full_name', 'college', 'fantasy_positions', 'active', 'swish_id',
       'news_updated', 'injury_notes', 'search_rank', 'birth_city',
       'practice_description'],
      dtype='object')

In [14]:
all_players = all_player_df[['full_name','fantasy_positions', 
                    'rotowire_id','fantasy_data_id', 'espn_id', 'sportradar_id',
                    'gsis_id','rotoworld_id', 'player_id', 'years_exp',
                    'pandascore_id']]

all_players = all_players.explode('fantasy_positions')
#all_players.head()

In [15]:
# Only Keeping Fantasy Relevant Positions
all_players = all_players[all_players['fantasy_positions'].isin(['QB','RB','WR','TE']) ]

In [16]:
# Cleansing out Player Names
all_players['full_name'] = all_players['full_name'].str.replace('[^a-zA-Z ]', '')
all_players['full_name'] = all_players.full_name.str.replace(r'(\bJr|Sr|III|II|I|V|IV|VI|VII|VIII|IX|X)$', '', regex=True).str.strip()
#all_players[all_players['full_name'].str.contains('Mark') == True]
# Removing Defenses
all_players = all_players[~ all_players['full_name'].isna()]

In [17]:
player_only = all_players[['player_id','full_name','fantasy_positions','years_exp']]
player_only

,player_id,full_name,fantasy_positions,years_exp
5870,5870,Daniel Jones,QB,2.0
1347,1347,Chris Gragg,TE,7.0
3396,3396,Charone Peake,WR,4.0
6343,6343,Manny Wilkins,QB,1.0
3199,3199,Michael Thomas,WR,5.0
...,...,...,...,...
7746,7746,Austin Trammell,WR,0.0
241,241,Spencer Larsen,RB,6.0
3684,3684,Derek Keaton,WR,0.0
7459,7459,Rashod Berry,TE,1.0


## Getting Draft Data from Sleeper

In [ ]:
# api-endpoint
URL = "https://api.sleeper.app/v1/draft/{}".format(draft_number)

# sending get request and saving the response as response object 
r_draft_league = requests.get(url = URL) 

# extracting data in json format 
draft_league = r_draft_league.json() 
#draft_league

In [ ]:
draft_league_df = pd.DataFrame.from_dict(draft_league, orient="index")
# Only Keeping Needed Columns
draft_league_df = draft_league_df.loc[['league_id', 'draft_id', 'season']]
#draft_league_df = draft_league_df.reset_index()
#draft_league_df.columns = ['field','value']
#draft_league_df = draft_league_df.pivot(columns='field', values='value')
# Transposing Data
draft_league_df = draft_league_df.T

cols = ['league_id', 'draft_id', 'season']
draft_league_df[cols] = draft_league_df[cols].applymap(np.int64)
#draft_league_df

## Getting Draft Pick Data

In [20]:
# api-endpoint
URL = "https://api.sleeper.app/v1/draft/{}/picks".format(draft_number)

# sending get request and saving the response as response object 
r_draft = requests.get(url = URL) 

# extracting data in json format 
draft_data = r_draft.json() 
draft_data[0]

{'round': 1,
 'roster_id': 5,
 'player_id': '4034',
 'picked_by': '341266941933056000',
 'pick_no': 1,
 'metadata': {'years_exp': '3',
  'team': 'CAR',
  'status': 'Active',
  'sport': 'nfl',
  'position': 'RB',
  'player_id': '4034',
  'number': '22',
  'news_updated': '1598384124944',
  'last_name': 'McCaffrey',
  'injury_status': '',
  'first_name': 'Christian'},
 'is_keeper': None,
 'draft_slot': 1,
 'draft_id': '591137395731107841'}

In [ ]:
all_draft_df = pd.DataFrame.from_dict(draft_data, orient="columns")
all_draft_df = all_draft_df[['round','roster_id','player_id','picked_by','pick_no','is_keeper','draft_id']]

cols = ['picked_by', 'draft_id']
all_draft_df[cols] = all_draft_df[cols].applymap(np.int64)

#all_draft_df.head()

#### Merging Draft Data with Picks

In [ ]:
all_draft_df  = pd.merge(all_draft_df , draft_league_df , how='left', on='draft_id')
#all_draft_df.head()

## Getting Player Transactions

In [23]:
# Calling API to retrieve transactions per week
raw_data = pd.DataFrame([])

# if using Midyear, limit range to number of weeks
#for i in range(1,18):
for i in range(1,15):
    r = requests.get("https://api.sleeper.app/v1/league/{}/transactions/{}".format(prev_league_id, i))
    # extracting data in json format 
    data = r.json()
    # getting list of adds per transaction id
    adds = pd.json_normalize(data = data,meta ='transaction_id', record_path = 'adds')
    adds.columns = ['player_id', 'transaction_id']
    # getting rest of json data
    df_temp = pd.DataFrame.from_dict(data , orient="columns")
    # Merging adds by transaction id
    df_temp = pd.merge(df_temp, adds, how = 'left', on = 'transaction_id')
    raw_data = raw_data.append(df_temp,ignore_index=True,sort=False)


In [24]:
# removing unnecessary columns
trans = raw_data[raw_data['status'] == 'complete' ]

trans = trans.loc[:,['player_id', 'type', 'leg',  'transaction_id']]
trans = trans[~trans['player_id'].isna()]

trans = trans.sort_values(["leg","transaction_id"]).groupby(["player_id","type"], 
                                                    as_index=False).first()
trans = trans.loc[:,['player_id', 'type', 'leg']]

trans_table = pd.crosstab(trans.player_id, trans.type, 
                          values = trans.leg,
                          margins = False,
                          aggfunc='sum').reset_index()
trans_table.head()

type,player_id,free_agent,trade,waiver
0,1048,10.0,NaN,NaN
1,1049,6.0,NaN,NaN
2,1067,NaN,NaN,4.0
3,1099,7.0,NaN,NaN
4,1144,14.0,NaN,NaN


In [25]:
# Merging with Player Data
player_only = pd.merge(player_only, trans_table, how='left', on='player_id')

In [26]:
player_only[player_only['trade']>0]

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver
4,3199,Michael Thomas,WR,5.0,NaN,11.0,NaN
128,1352,Robert Woods,WR,8.0,NaN,1.0,NaN
799,5872,Deebo Samuel,WR,2.0,NaN,1.0,NaN
914,5170,Phillip Lindsay,RB,3.0,NaN,1.0,4.0
1383,536,Antonio Brown,WR,11.0,7.0,1.0,13.0
1638,6806,JK Dobbins,RB,1.0,NaN,10.0,7.0
1668,5846,DK Metcalf,WR,2.0,NaN,11.0,NaN
2102,4098,Kareem Hunt,RB,4.0,NaN,1.0,NaN
2238,1476,Latavius Murray,RB,8.0,NaN,5.0,11.0
2398,2216,Mike Evans,WR,7.0,NaN,1.0,NaN


# Getting Data into Single Dataframe

In [27]:
df = pd.merge(player_only, all_draft_df, how='left', on='player_id')
# getting rid of nulls
df['draft_id'] = df['draft_id'].fillna(0)
# formmatting draft id
df['draft_id'] = df['draft_id'].astype('int64')
#df['draft_id'] = df['draft_id'].str.strip()
df

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver,round,roster_id,picked_by,pick_no,is_keeper,draft_id,league_id,season
0,5870,Daniel Jones,QB,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
1,1347,Chris Gragg,TE,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
2,3396,Charone Peake,WR,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
3,6343,Manny Wilkins,QB,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
4,3199,Michael Thomas,WR,5.0,NaN,11.0,NaN,6.0,7.0,3.411054e+17,51.0,True,591137395731107840,5.911374e+17,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2772,7746,Austin Trammell,WR,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
2773,241,Spencer Larsen,RB,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
2774,3684,Derek Keaton,WR,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
2775,7459,Rashod Berry,TE,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


In [ ]:
# merging with trade data
#df = pd.merge(df, all_trade_df, how='left', on= ['round','roster_id', 'draft_id'])
# Creating new Roster ID to account for draft pick trading
#df['drafted_roster_id'] = df['owner_id'].fillna(df['roster_id'])
# dropping columns no longer needed
#df = df.drop(['previous_owner_id','owner_id', 'roster_id', 'season'], axis=1)


# merging with owners to get drafted by
#df =  pd.merge(df, owners, how='left',
                  #left_on = ['roster_id', 'league_id'], 
                  #right_on = ['roster_id', 'league_id'])

# merging with owners to get drafted by
df =  pd.merge(df, owners, how='left',
                  left_on = ['roster_id'], 
                  right_on = ['roster_id'])


# dropping columns no longer needed
df['draft_owner'] = df['display_name']
df['draft_owner_id'] = df['owner_id']
df['draft_roster_id'] = df['roster_id']
df = df.drop(['picked_by', 'display_name', 'owner_id', 'roster_id'], axis=1)

#df

In [ ]:
# Need to INNER JOIN on Rosters, those are the only keepable players
# Getting current owner info
df =  pd.merge(df, rosters, how='inner', 
                left_on = ['player_id'], 
                right_on = ['players'])
df = df.drop(['league_id_x','league_id_y'], axis=1)

# changing column names and dropping ones no longer needed
df['current_owner'] = df['display_name']
df['current_owner_id'] = df['owner_id']
df['current_roster_id'] = df['roster_id']
df = df.drop(['display_name', 'owner_id', 'roster_id', 'players'], axis=1)

#df

In [30]:
df

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver,round,pick_no,is_keeper,draft_id,season,draft_owner,draft_owner_id,draft_roster_id,league_id,current_owner,current_owner_id,current_roster_id
0,3199,Michael Thomas,WR,5.0,NaN,11.0,NaN,6.0,51.0,True,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,Conorrich,341015344539787264,4
1,4981,Calvin Ridley,WR,3.0,NaN,NaN,NaN,3.0,21.0,None,591137395731107840,2020.0,9johnb,3.412669e+17,5.0,722951233224155136,9johnb,341266941933056000,5
2,4036,Corey Davis,WR,4.0,2.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,722951233224155136,Rioman,341705565078040576,10
3,1339,Zach Ertz,TE,8.0,NaN,NaN,NaN,3.0,26.0,None,591137395731107840,2020.0,Conorrich,3.410153e+17,4.0,722951233224155136,Conorrich,341015344539787264,4
4,954,Cam Newton,QB,10.0,13.0,NaN,11.0,14.0,131.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,MAPavoloni,340924015470219264,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2382,Duke Johnson,RB,6.0,14.0,NaN,4.0,14.0,133.0,None,591137395731107840,2020.0,tweet011,3.413494e+17,9.0,722951233224155136,Conorrich,341015344539787264,4
140,6130,Devin Singletary,RB,2.0,NaN,NaN,NaN,11.0,102.0,None,591137395731107840,2020.0,tyso0042,3.409441e+17,2.0,722951233224155136,tyso0042,340944113622892544,2
141,6797,Justin Herbert,QB,1.0,NaN,NaN,6.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,722951233224155136,tweet011,341349429489446912,9
142,4273,Chris Carson,RB,4.0,NaN,NaN,NaN,1.0,10.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,magicpajcic,341105417952202752,7


In [31]:
df[df['round'] == 2 ]

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver,round,pick_no,is_keeper,draft_id,season,draft_owner,draft_owner_id,draft_roster_id,league_id,current_owner,current_owner_id,current_roster_id
33,1466,Travis Kelce,TE,8.0,NaN,NaN,NaN,2.0,13.0,None,591137395731107840,2020.0,tweet011,3.413494e+17,9.0,722951233224155136,tweet011,341349429489446912,9
36,2309,Amari Cooper,WR,6.0,NaN,NaN,NaN,2.0,17.0,None,591137395731107840,2020.0,kiwirado,3.412701e+17,6.0,722951233224155136,kiwirado,341270051682283520,6
41,4037,Chris Godwin,WR,4.0,NaN,NaN,NaN,2.0,12.0,None,591137395731107840,2020.0,FillJoo,3.413478e+17,8.0,722951233224155136,FillJoo,341347820613173248,8
50,6813,Jonathan Taylor,RB,1.0,NaN,NaN,NaN,2.0,18.0,None,591137395731107840,2020.0,brisbois19,3.409464e+17,3.0,722951233224155136,kiwirado,341270051682283520,6
52,4881,Lamar Jackson,QB,3.0,NaN,NaN,NaN,2.0,14.0,None,591137395731107840,2020.0,MAPavoloni,3.409240e+17,1.0,722951233224155136,MAPavoloni,340924015470219264,1
61,2315,Todd Gurley,RB,6.0,NaN,NaN,NaN,2.0,16.0,None,591137395731107840,2020.0,Rioman,3.417056e+17,10.0,722951233224155136,Rioman,341705565078040576,10
82,1426,DeAndre Hopkins,WR,8.0,NaN,NaN,NaN,2.0,11.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,magicpajcic,341105417952202752,7
83,1689,Adam Thielen,WR,8.0,NaN,NaN,NaN,2.0,15.0,None,591137395731107840,2020.0,Conorrich,3.410153e+17,4.0,722951233224155136,Conorrich,341015344539787264,4
111,4983,DJ Moore,WR,3.0,NaN,NaN,NaN,2.0,20.0,None,591137395731107840,2020.0,9johnb,3.412669e+17,5.0,722951233224155136,9johnb,341266941933056000,5
132,1408,LeVeon Bell,RB,8.0,NaN,NaN,10.0,2.0,19.0,None,591137395731107840,2020.0,tyso0042,3.409441e+17,2.0,722951233224155136,brisbois19,340946372834410496,3


In [32]:
df[df['full_name']=='Darrell Henderson']

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver,round,pick_no,is_keeper,draft_id,season,draft_owner,draft_owner_id,draft_roster_id,league_id,current_owner,current_owner_id,current_roster_id
87,5916,Darrell Henderson,RB,2.0,NaN,NaN,NaN,16.0,155.0,None,591137395731107840,2020.0,Conorrich,3.410153e+17,4.0,722951233224155136,MAPavoloni,340924015470219264,1


In [33]:
# getting rid of undrafted players or not on roster
#df = df.drop(df[(df['draft_roster_id'].isna()) | (df['current_roster_id'].isna())].index)
indexNames = df[ (df['draft_owner_id'].isna()) & (df['current_owner_id'].isna()) ].index
df.drop(indexNames , inplace=True)
df

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver,round,pick_no,is_keeper,draft_id,season,draft_owner,draft_owner_id,draft_roster_id,league_id,current_owner,current_owner_id,current_roster_id
0,3199,Michael Thomas,WR,5.0,NaN,11.0,NaN,6.0,51.0,True,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,Conorrich,341015344539787264,4
1,4981,Calvin Ridley,WR,3.0,NaN,NaN,NaN,3.0,21.0,None,591137395731107840,2020.0,9johnb,3.412669e+17,5.0,722951233224155136,9johnb,341266941933056000,5
2,4036,Corey Davis,WR,4.0,2.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,722951233224155136,Rioman,341705565078040576,10
3,1339,Zach Ertz,TE,8.0,NaN,NaN,NaN,3.0,26.0,None,591137395731107840,2020.0,Conorrich,3.410153e+17,4.0,722951233224155136,Conorrich,341015344539787264,4
4,954,Cam Newton,QB,10.0,13.0,NaN,11.0,14.0,131.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,MAPavoloni,340924015470219264,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2382,Duke Johnson,RB,6.0,14.0,NaN,4.0,14.0,133.0,None,591137395731107840,2020.0,tweet011,3.413494e+17,9.0,722951233224155136,Conorrich,341015344539787264,4
140,6130,Devin Singletary,RB,2.0,NaN,NaN,NaN,11.0,102.0,None,591137395731107840,2020.0,tyso0042,3.409441e+17,2.0,722951233224155136,tyso0042,340944113622892544,2
141,6797,Justin Herbert,QB,1.0,NaN,NaN,6.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,722951233224155136,tweet011,341349429489446912,9
142,4273,Chris Carson,RB,4.0,NaN,NaN,NaN,1.0,10.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,magicpajcic,341105417952202752,7


In [34]:
# indicator if player kept throughout the season
df['held_player_flag'] = np.where(df['draft_roster_id'] == df['current_roster_id'], 1, 0)

# Free Agent Indicator
df['free_agent_flag'] = np.where(df['draft_roster_id'].isna(), 1, 0)

# Traded Player
df['traded_player_flag'] = df[['free_agent','trade','waiver']].max(axis=1)
df['traded_player_flag'] = np.where(df['traded_player_flag'] == df['trade'] , 1, 0)
df

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver,round,pick_no,is_keeper,draft_id,season,draft_owner,draft_owner_id,draft_roster_id,league_id,current_owner,current_owner_id,current_roster_id,held_player_flag,free_agent_flag,traded_player_flag
0,3199,Michael Thomas,WR,5.0,NaN,11.0,NaN,6.0,51.0,True,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,Conorrich,341015344539787264,4,0,0,1
1,4981,Calvin Ridley,WR,3.0,NaN,NaN,NaN,3.0,21.0,None,591137395731107840,2020.0,9johnb,3.412669e+17,5.0,722951233224155136,9johnb,341266941933056000,5,1,0,0
2,4036,Corey Davis,WR,4.0,2.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,722951233224155136,Rioman,341705565078040576,10,0,1,0
3,1339,Zach Ertz,TE,8.0,NaN,NaN,NaN,3.0,26.0,None,591137395731107840,2020.0,Conorrich,3.410153e+17,4.0,722951233224155136,Conorrich,341015344539787264,4,1,0,0
4,954,Cam Newton,QB,10.0,13.0,NaN,11.0,14.0,131.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,MAPavoloni,340924015470219264,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2382,Duke Johnson,RB,6.0,14.0,NaN,4.0,14.0,133.0,None,591137395731107840,2020.0,tweet011,3.413494e+17,9.0,722951233224155136,Conorrich,341015344539787264,4,0,0,0
140,6130,Devin Singletary,RB,2.0,NaN,NaN,NaN,11.0,102.0,None,591137395731107840,2020.0,tyso0042,3.409441e+17,2.0,722951233224155136,tyso0042,340944113622892544,2,1,0,0
141,6797,Justin Herbert,QB,1.0,NaN,NaN,6.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,722951233224155136,tweet011,341349429489446912,9,0,1,0
142,4273,Chris Carson,RB,4.0,NaN,NaN,NaN,1.0,10.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,magicpajcic,341105417952202752,7,1,0,0


In [35]:
#df[df['free_agent_flag'] == 1]
df[df['traded_player_flag'] == 1]

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver,round,pick_no,is_keeper,draft_id,season,draft_owner,draft_owner_id,draft_roster_id,league_id,current_owner,current_owner_id,current_roster_id,held_player_flag,free_agent_flag,traded_player_flag
0,3199,Michael Thomas,WR,5.0,NaN,11.0,NaN,6.0,51.0,True,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,Conorrich,341015344539787264,4,0,0,1
6,1352,Robert Woods,WR,8.0,NaN,1.0,NaN,8.0,78.0,True,591137395731107840,2020.0,brisbois19,3.409464e+17,3.0,722951233224155136,brisbois19,340946372834410496,3,1,0,1
40,5872,Deebo Samuel,WR,2.0,NaN,1.0,NaN,8.0,73.0,None,591137395731107840,2020.0,tweet011,3.413494e+17,9.0,722951233224155136,9johnb,341266941933056000,5,0,0,1
89,6806,JK Dobbins,RB,1.0,NaN,10.0,7.0,5.0,43.0,None,591137395731107840,2020.0,brisbois19,3.409464e+17,3.0,722951233224155136,tyso0042,340944113622892544,2,0,0,1
90,5846,DK Metcalf,WR,2.0,NaN,11.0,NaN,3.0,30.0,None,591137395731107840,2020.0,MAPavoloni,3.409240e+17,1.0,722951233224155136,magicpajcic,341105417952202752,7,0,0,1
118,4098,Kareem Hunt,RB,4.0,NaN,1.0,NaN,10.0,98.0,True,591137395731107840,2020.0,brisbois19,3.409464e+17,3.0,722951233224155136,brisbois19,340946372834410496,3,1,0,1
124,2216,Mike Evans,WR,7.0,NaN,1.0,NaN,7.0,64.0,True,591137395731107840,2020.0,kiwirado,3.412701e+17,6.0,722951233224155136,kiwirado,341270051682283520,6,1,0,1
131,4040,JuJu SmithSchuster,WR,4.0,NaN,10.0,NaN,7.0,62.0,True,591137395731107840,2020.0,tyso0042,3.409441e+17,2.0,722951233224155136,9johnb,341266941933056000,5,0,0,1


# Need to figure out how to incorporate trades for keepers made during off season. Could look at historical keeper file, add in the owner name, and join to show vs current owner and get trades that way

In [36]:
# adding historical keeper data 
hist_keepers = pd.read_csv('C:/Users/Patrick/HistoricalKeepers_{}_{}.csv'.format(commissioner, season))

# Merging with existing data
df = pd.merge(df, hist_keepers, how='left', on='full_name')
df

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver,round,pick_no,is_keeper,draft_id,season,draft_owner,draft_owner_id,draft_roster_id,league_id,current_owner,current_owner_id,current_roster_id,held_player_flag,free_agent_flag,traded_player_flag,last_draft_year,last_draft_owner,years_kept_same_owner
0,3199,Michael Thomas,WR,5.0,NaN,11.0,NaN,6.0,51.0,True,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,Conorrich,341015344539787264,4,0,0,1,2020.0,magicpajcic,4.0
1,4981,Calvin Ridley,WR,3.0,NaN,NaN,NaN,3.0,21.0,None,591137395731107840,2020.0,9johnb,3.412669e+17,5.0,722951233224155136,9johnb,341266941933056000,5,1,0,0,NaN,NaN,NaN
2,4036,Corey Davis,WR,4.0,2.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,722951233224155136,Rioman,341705565078040576,10,0,1,0,NaN,NaN,NaN
3,1339,Zach Ertz,TE,8.0,NaN,NaN,NaN,3.0,26.0,None,591137395731107840,2020.0,Conorrich,3.410153e+17,4.0,722951233224155136,Conorrich,341015344539787264,4,1,0,0,NaN,NaN,NaN
4,954,Cam Newton,QB,10.0,13.0,NaN,11.0,14.0,131.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,MAPavoloni,340924015470219264,1,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2382,Duke Johnson,RB,6.0,14.0,NaN,4.0,14.0,133.0,None,591137395731107840,2020.0,tweet011,3.413494e+17,9.0,722951233224155136,Conorrich,341015344539787264,4,0,0,0,NaN,NaN,NaN
140,6130,Devin Singletary,RB,2.0,NaN,NaN,NaN,11.0,102.0,None,591137395731107840,2020.0,tyso0042,3.409441e+17,2.0,722951233224155136,tyso0042,340944113622892544,2,1,0,0,2020.0,tyso0042,1.0
141,6797,Justin Herbert,QB,1.0,NaN,NaN,6.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,722951233224155136,tweet011,341349429489446912,9,0,1,0,NaN,NaN,NaN
142,4273,Chris Carson,RB,4.0,NaN,NaN,NaN,1.0,10.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,magicpajcic,341105417952202752,7,1,0,0,NaN,NaN,NaN


In [37]:
df[df['round']==2]

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver,round,pick_no,is_keeper,draft_id,season,draft_owner,draft_owner_id,draft_roster_id,league_id,current_owner,current_owner_id,current_roster_id,held_player_flag,free_agent_flag,traded_player_flag,last_draft_year,last_draft_owner,years_kept_same_owner
33,1466,Travis Kelce,TE,8.0,NaN,NaN,NaN,2.0,13.0,None,591137395731107840,2020.0,tweet011,3.413494e+17,9.0,722951233224155136,tweet011,341349429489446912,9,1,0,0,2020.0,brisbois19,2.0
36,2309,Amari Cooper,WR,6.0,NaN,NaN,NaN,2.0,17.0,None,591137395731107840,2020.0,kiwirado,3.412701e+17,6.0,722951233224155136,kiwirado,341270051682283520,6,1,0,0,NaN,NaN,NaN
41,4037,Chris Godwin,WR,4.0,NaN,NaN,NaN,2.0,12.0,None,591137395731107840,2020.0,FillJoo,3.413478e+17,8.0,722951233224155136,FillJoo,341347820613173248,8,1,0,0,NaN,NaN,NaN
50,6813,Jonathan Taylor,RB,1.0,NaN,NaN,NaN,2.0,18.0,None,591137395731107840,2020.0,brisbois19,3.409464e+17,3.0,722951233224155136,kiwirado,341270051682283520,6,0,0,0,NaN,NaN,NaN
52,4881,Lamar Jackson,QB,3.0,NaN,NaN,NaN,2.0,14.0,None,591137395731107840,2020.0,MAPavoloni,3.409240e+17,1.0,722951233224155136,MAPavoloni,340924015470219264,1,1,0,0,NaN,NaN,NaN
61,2315,Todd Gurley,RB,6.0,NaN,NaN,NaN,2.0,16.0,None,591137395731107840,2020.0,Rioman,3.417056e+17,10.0,722951233224155136,Rioman,341705565078040576,10,1,0,0,NaN,NaN,NaN
82,1426,DeAndre Hopkins,WR,8.0,NaN,NaN,NaN,2.0,11.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,magicpajcic,341105417952202752,7,1,0,0,NaN,NaN,NaN
83,1689,Adam Thielen,WR,8.0,NaN,NaN,NaN,2.0,15.0,None,591137395731107840,2020.0,Conorrich,3.410153e+17,4.0,722951233224155136,Conorrich,341015344539787264,4,1,0,0,NaN,NaN,NaN
111,4983,DJ Moore,WR,3.0,NaN,NaN,NaN,2.0,20.0,None,591137395731107840,2020.0,9johnb,3.412669e+17,5.0,722951233224155136,9johnb,341266941933056000,5,1,0,0,NaN,NaN,NaN
132,1408,LeVeon Bell,RB,8.0,NaN,NaN,10.0,2.0,19.0,None,591137395731107840,2020.0,tyso0042,3.409441e+17,2.0,722951233224155136,brisbois19,340946372834410496,3,0,0,0,NaN,NaN,NaN


### Building out Keeper logic

In [38]:
df['keeper_round'] = np.nan

# Specify Which League Keeper Logic to Use

In [39]:
df.head()

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver,round,pick_no,is_keeper,draft_id,season,draft_owner,draft_owner_id,draft_roster_id,league_id,current_owner,current_owner_id,current_roster_id,held_player_flag,free_agent_flag,traded_player_flag,last_draft_year,last_draft_owner,years_kept_same_owner,keeper_round
0,3199,Michael Thomas,WR,5.0,NaN,11.0,NaN,6.0,51.0,True,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,Conorrich,341015344539787264,4,0,0,1,2020.0,magicpajcic,4.0,NaN
1,4981,Calvin Ridley,WR,3.0,NaN,NaN,NaN,3.0,21.0,None,591137395731107840,2020.0,9johnb,3.412669e+17,5.0,722951233224155136,9johnb,341266941933056000,5,1,0,0,NaN,NaN,NaN,NaN
2,4036,Corey Davis,WR,4.0,2.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,722951233224155136,Rioman,341705565078040576,10,0,1,0,NaN,NaN,NaN,NaN
3,1339,Zach Ertz,TE,8.0,NaN,NaN,NaN,3.0,26.0,None,591137395731107840,2020.0,Conorrich,3.410153e+17,4.0,722951233224155136,Conorrich,341015344539787264,4,1,0,0,NaN,NaN,NaN,NaN
4,954,Cam Newton,QB,10.0,13.0,NaN,11.0,14.0,131.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,MAPavoloni,340924015470219264,1,0,0,0,NaN,NaN,NaN,NaN


In [40]:
df[ (df['current_owner'] != df['draft_owner']) & ~(df['draft_owner'].isna() )  ] 
#= df['round']

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver,round,pick_no,is_keeper,draft_id,season,draft_owner,draft_owner_id,draft_roster_id,league_id,current_owner,current_owner_id,current_roster_id,held_player_flag,free_agent_flag,traded_player_flag,last_draft_year,last_draft_owner,years_kept_same_owner,keeper_round
0,3199,Michael Thomas,WR,5.0,NaN,11.0,NaN,6.0,51.0,True,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,Conorrich,341015344539787264,4,0,0,1,2020.0,magicpajcic,4.0,NaN
4,954,Cam Newton,QB,10.0,13.0,NaN,11.0,14.0,131.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,MAPavoloni,340924015470219264,1,0,0,0,NaN,NaN,NaN,NaN
5,6845,Zack Moss,RB,1.0,8.0,NaN,NaN,6.0,54.0,None,591137395731107840,2020.0,MAPavoloni,3.409240e+17,1.0,722951233224155136,brisbois19,340946372834410496,3,0,0,0,NaN,NaN,NaN,NaN
7,5967,Tony Pollard,RB,2.0,NaN,NaN,NaN,13.0,129.0,None,591137395731107840,2020.0,FillJoo,3.413478e+17,8.0,722951233224155136,brisbois19,340946372834410496,3,0,0,0,NaN,NaN,NaN,NaN
32,1067,Marvin Jones,WR,9.0,NaN,NaN,4.0,9.0,84.0,None,591137395731107840,2020.0,brisbois19,3.409464e+17,3.0,722951233224155136,FillJoo,341347820613173248,8,0,0,0,NaN,NaN,NaN,NaN
40,5872,Deebo Samuel,WR,2.0,NaN,1.0,NaN,8.0,73.0,None,591137395731107840,2020.0,tweet011,3.413494e+17,9.0,722951233224155136,9johnb,341266941933056000,5,0,0,1,NaN,NaN,NaN,NaN
45,6878,Anthony McFarland,RB,1.0,5.0,NaN,NaN,14.0,140.0,None,591137395731107840,2020.0,9johnb,3.412669e+17,5.0,722951233224155136,kiwirado,341270051682283520,6,0,0,0,NaN,NaN,NaN,NaN
50,6813,Jonathan Taylor,RB,1.0,NaN,NaN,NaN,2.0,18.0,None,591137395731107840,2020.0,brisbois19,3.409464e+17,3.0,722951233224155136,kiwirado,341270051682283520,6,0,0,0,NaN,NaN,NaN,NaN
58,6828,AJ Dillon,RB,1.0,14.0,NaN,NaN,12.0,113.0,None,591137395731107840,2020.0,tweet011,3.413494e+17,9.0,722951233224155136,tyso0042,340944113622892544,2,0,0,0,NaN,NaN,NaN,NaN
60,5038,Michael Gallup,WR,3.0,14.0,NaN,NaN,15.0,145.0,True,591137395731107840,2020.0,Rioman,3.417056e+17,10.0,722951233224155136,MAPavoloni,340924015470219264,1,0,0,0,2020.0,Rioman,1.0,NaN


In [41]:
# Marco's League Logic
df['keeper_round'][ df['held_player_flag'] == 1 ] = df['round'] - (df['years_kept_same_owner'].fillna(0) + 1)
df['keeper_round'][ df['traded_player_flag'] == 1 ] = df['round']
df['keeper_round'][ df['free_agent_flag'] == 1 ] = 12

# getting players traded in off-season
df['keeper_round'][ df['free_agent_flag'] == 1 ] = 12

df['keeper_round'][ (df['held_player_flag'] == 0) & 
                   (df['traded_player_flag'] == 0) & 
                   (df['free_agent_flag'] == 0) &  
                   (df['years_kept_same_owner'].isna()) ]  = 12

C:\Users\Patrick\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Patrick\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Patrick\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after remo

In [42]:
df[df['full_name'].str.contains('Taylor') == True]

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver,round,pick_no,is_keeper,draft_id,season,draft_owner,draft_owner_id,draft_roster_id,league_id,current_owner,current_owner_id,current_roster_id,held_player_flag,free_agent_flag,traded_player_flag,last_draft_year,last_draft_owner,years_kept_same_owner,keeper_round
50,6813,Jonathan Taylor,RB,1.0,NaN,NaN,NaN,2.0,18.0,None,591137395731107840,2020.0,brisbois19,3.409464e+17,3.0,722951233224155136,kiwirado,341270051682283520,6,0,0,0,NaN,NaN,NaN,12.0


#### Old logic - pre excel input
df['keeper_round'] = np.where(df.held_player_flag == 1, df['round'] - 2, 
                              np.where(df.traded_player_flag == 1, df['round'], 10))

In [43]:
df

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver,round,pick_no,is_keeper,draft_id,season,draft_owner,draft_owner_id,draft_roster_id,league_id,current_owner,current_owner_id,current_roster_id,held_player_flag,free_agent_flag,traded_player_flag,last_draft_year,last_draft_owner,years_kept_same_owner,keeper_round
0,3199,Michael Thomas,WR,5.0,NaN,11.0,NaN,6.0,51.0,True,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,Conorrich,341015344539787264,4,0,0,1,2020.0,magicpajcic,4.0,6.0
1,4981,Calvin Ridley,WR,3.0,NaN,NaN,NaN,3.0,21.0,None,591137395731107840,2020.0,9johnb,3.412669e+17,5.0,722951233224155136,9johnb,341266941933056000,5,1,0,0,NaN,NaN,NaN,2.0
2,4036,Corey Davis,WR,4.0,2.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,722951233224155136,Rioman,341705565078040576,10,0,1,0,NaN,NaN,NaN,12.0
3,1339,Zach Ertz,TE,8.0,NaN,NaN,NaN,3.0,26.0,None,591137395731107840,2020.0,Conorrich,3.410153e+17,4.0,722951233224155136,Conorrich,341015344539787264,4,1,0,0,NaN,NaN,NaN,2.0
4,954,Cam Newton,QB,10.0,13.0,NaN,11.0,14.0,131.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,MAPavoloni,340924015470219264,1,0,0,0,NaN,NaN,NaN,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2382,Duke Johnson,RB,6.0,14.0,NaN,4.0,14.0,133.0,None,591137395731107840,2020.0,tweet011,3.413494e+17,9.0,722951233224155136,Conorrich,341015344539787264,4,0,0,0,NaN,NaN,NaN,12.0
140,6130,Devin Singletary,RB,2.0,NaN,NaN,NaN,11.0,102.0,None,591137395731107840,2020.0,tyso0042,3.409441e+17,2.0,722951233224155136,tyso0042,340944113622892544,2,1,0,0,2020.0,tyso0042,1.0,9.0
141,6797,Justin Herbert,QB,1.0,NaN,NaN,6.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,722951233224155136,tweet011,341349429489446912,9,0,1,0,NaN,NaN,NaN,12.0
142,4273,Chris Carson,RB,4.0,NaN,NaN,NaN,1.0,10.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,magicpajcic,341105417952202752,7,1,0,0,NaN,NaN,NaN,0.0


In [44]:
df[ ~df['keeper_round'].isna()]

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver,round,pick_no,is_keeper,draft_id,season,draft_owner,draft_owner_id,draft_roster_id,league_id,current_owner,current_owner_id,current_roster_id,held_player_flag,free_agent_flag,traded_player_flag,last_draft_year,last_draft_owner,years_kept_same_owner,keeper_round
0,3199,Michael Thomas,WR,5.0,NaN,11.0,NaN,6.0,51.0,True,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,Conorrich,341015344539787264,4,0,0,1,2020.0,magicpajcic,4.0,6.0
1,4981,Calvin Ridley,WR,3.0,NaN,NaN,NaN,3.0,21.0,None,591137395731107840,2020.0,9johnb,3.412669e+17,5.0,722951233224155136,9johnb,341266941933056000,5,1,0,0,NaN,NaN,NaN,2.0
2,4036,Corey Davis,WR,4.0,2.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,722951233224155136,Rioman,341705565078040576,10,0,1,0,NaN,NaN,NaN,12.0
3,1339,Zach Ertz,TE,8.0,NaN,NaN,NaN,3.0,26.0,None,591137395731107840,2020.0,Conorrich,3.410153e+17,4.0,722951233224155136,Conorrich,341015344539787264,4,1,0,0,NaN,NaN,NaN,2.0
4,954,Cam Newton,QB,10.0,13.0,NaN,11.0,14.0,131.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,MAPavoloni,340924015470219264,1,0,0,0,NaN,NaN,NaN,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2382,Duke Johnson,RB,6.0,14.0,NaN,4.0,14.0,133.0,None,591137395731107840,2020.0,tweet011,3.413494e+17,9.0,722951233224155136,Conorrich,341015344539787264,4,0,0,0,NaN,NaN,NaN,12.0
140,6130,Devin Singletary,RB,2.0,NaN,NaN,NaN,11.0,102.0,None,591137395731107840,2020.0,tyso0042,3.409441e+17,2.0,722951233224155136,tyso0042,340944113622892544,2,1,0,0,2020.0,tyso0042,1.0,9.0
141,6797,Justin Herbert,QB,1.0,NaN,NaN,6.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,722951233224155136,tweet011,341349429489446912,9,0,1,0,NaN,NaN,NaN,12.0
142,4273,Chris Carson,RB,4.0,NaN,NaN,NaN,1.0,10.0,None,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,magicpajcic,341105417952202752,7,1,0,0,NaN,NaN,NaN,0.0


## Fantasy Football Calculator
https://fantasyfootballcalculator.com/

In [45]:
r = requests.get("https://fantasyfootballcalculator.com/api/v1/adp/half-ppr?teams={}&year={}".format(num_teams, season))

# extracting data in json format 
fb_calc = r.json() 

x = r.json()['players'][:]

In [46]:
fb_calc = pd.DataFrame(x)

fb_calc = fb_calc[[ 'name', 'adp']]
fb_calc.columns = ['Player_Name', 'FBCalc_adp']

# Cleansing out Player Names
fb_calc['Player_Name'] = fb_calc['Player_Name'].str.replace('[^a-zA-Z ]', '')
fb_calc['Player_Name'] = fb_calc.Player_Name.str.replace(r'(\bJr|Sr|III|II|I|V|IV|VI|VII|VIII|IX|X)$', '', regex=True).str.strip()

# Replacing "Pat" Names
fb_calc['Player_Name'] = fb_calc['Player_Name'].str.replace('Pat ', 'Patrick ')

In [47]:
fb_calc[fb_calc['Player_Name'].str.startswith('Pat')]

,Player_Name,FBCalc_adp
21,Patrick Mahomes,21.5
175,Patrick Freiermuth,156.2


### Scraping, doesn't work on FBG

#url = 'https://subscribers.footballguys.com/apps/draftlist.php?t=0&ppr=1'
url = 'https://subscribers.footballguys.com/apps/adp-ppr.php?viewpos=all&sortby=consensus'
html = requests.get(url, auth=('tyso0042@gmail.com', 'bonerjams010'))
html

from bs4 import BeautifulSoup

soup = BeautifulSoup(html.text)

data = []
#table = soup.find('table', attrs={'class':'lineItemsTable'})
table = soup.find('table')

table_body = table.find('tbody')

table_body

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

### Go to websites, download csvs and read into program
* https://www.fantasypros.com/nfl/adp/ppr-overall.php

* https://www.fantasypros.com/nfl/rankings/dynasty-overall.php

* https://www.rotoviz.com/dynasty-adp/

## Fantasy Pros ADP Data

In [48]:
fp_adp = pd.read_csv("C:/Users/Patrick/FantasyPros_{}_Overall_ADP_Rankings.csv".format(season))
                     
# Check if data pulls in for NaN Columns
drop_cols = ['Rank','Bye','AVG']
fp_adp = fp_adp.drop(columns = drop_cols)

# Renaming columns
colnames = ['Player_Name','Team','POS','FP_ESPN','FP_RTSports','FP_MFL','FP_Fantrax', 'FP_FFC', 'FP_Sleeper']
fp_adp.columns = colnames

# Cleansing out Player Names
fp_adp['Player_Name'] = fp_adp['Player_Name'].str.replace('[^a-zA-Z ]', '')
fp_adp['Player_Name'] = fp_adp.Player_Name.str.replace(r'(\bJr|Sr|III|II|I|V|IV|VI|VII|VIII|IX|X)$', '', regex=True).str.strip()
fp_adp['Player_Name']
fp_adp.head()

,Player_Name,Team,POS,FP_ESPN,FP_RTSports,FP_MFL,FP_Fantrax,FP_FFC,FP_Sleeper
0,Christian McCaffrey,CAR,RB1,1.0,1.0,1.0,1.0,1.0,1.0
1,Dalvin Cook,MIN,RB2,2.0,2.0,2.0,2.0,2.0,2.0
2,Alvin Kamara,NO,RB3,3.0,3.0,3.0,3.0,3.0,3.0
3,Derrick Henry,TEN,RB4,4.0,4.0,4.0,4.0,5.0,4.0
4,Ezekiel Elliott,DAL,RB5,8.0,5.0,5.0,5.0,4.0,5.0


## Fantasy Pros Dynasty

In [49]:
fp_dyn = pd.read_csv("C:/Users/Patrick/FantasyPros_{}_Dynasty_ALL_Rankings.csv".format(season))
fp_dyn

,RK,TIERS,PLAYER NAME,TEAM,POS,BEST,WORST,AVG.,STD.DEV
0,1,1,Christian McCaffrey,CAR,RB1,1,2,1.1,0.3
1,2,1,Jonathan Taylor,IND,RB2,1,11,4.0,2.4
2,3,1,Dalvin Cook,MIN,RB3,2,12,4.2,2.6
3,4,1,Saquon Barkley,NYG,RB4,2,14,4.6,3.2
4,5,2,Alvin Kamara,NO,RB5,3,14,7.1,3.0
...,...,...,...,...,...,...,...,...,...
522,523,16,Willie Snead IV,LV,WR187,405,505,466.0,43.7
523,524,16,Trey Quinn,FA,WR188,406,506,467.0,43.7
524,525,16,Theo Riddick,LV,RB138,408,507,468.3,43.2
525,526,16,Nick Mullens,CLE,QB61,429,485,457.0,28.0


In [50]:
fp_dyn = pd.read_csv("C:/Users/Patrick/FantasyPros_{}_Dynasty_ALL_Rankings.csv".format(season))
drop_cols = ['RK','TIERS','TEAM','POS','BEST','WORST','STD.DEV']
fp_dyn = fp_dyn.drop(columns = drop_cols)
colnames = ['Player_Name','FP_Dyn_Avg']
fp_dyn.columns = colnames

# Cleansing out Player Names
fp_dyn['Player_Name'] = fp_dyn['Player_Name'].str.replace('[^a-zA-Z ]', '')
fp_dyn['Player_Name'] = fp_dyn.Player_Name.str.replace(r'(\bJr|Sr|III|II|I|V|IV|VI|VII|VIII|IX|X)$', '', regex=True).str.strip()

fp_dyn

,Player_Name,FP_Dyn_Avg
0,Christian McCaffrey,1.1
1,Jonathan Taylor,4.0
2,Dalvin Cook,4.2
3,Saquon Barkley,4.6
4,Alvin Kamara,7.1
...,...,...
522,Willie Snead,466.0
523,Trey Quinn,467.0
524,Theo Riddick,468.3
525,Nick Mullens,457.0


## Rotoviz Dynasty Data

In [51]:
rv_date = '2021-09-03'

rv_dyn = pd.read_csv("C:/Users/Patrick/DynastyADP_MyFantasyLeaguecom_StartUp_1QB_{}.csv".format(rv_date) )
drop_cols = ['Rank','Drafts','PosADP','RVRank','RVPosRk','Earliest','Latest','StdDev']
rv_dyn = rv_dyn.drop(columns = drop_cols)

colnames = ['Player_Name','Pos','Team','RV_Dyn_Avg']
rv_dyn.columns = colnames

# Cleansing out Player Names
rv_dyn['Player_Name'] = rv_dyn['Player_Name'].str.replace('[^a-zA-Z ]', '')
rv_dyn['Player_Name'] = rv_dyn.Player_Name.str.replace(r'(\bJr|Sr|III|II|I|V|IV|VI|VII|VIII|IX|X)$', '', regex=True).str.strip()
rv_dyn['Player_Name']

rv_dyn

,Player_Name,Pos,Team,RV_Dyn_Avg
0,Christian McCaffrey,RB,CAR,1.78
1,Dalvin Cook,RB,MIN,4.25
2,Jonathan Taylor,RB,IND,5.54
3,Alvin Kamara,RB,NOS,5.64
4,Saquon Barkley,RB,NYG,7.24
...,...,...,...,...
330,Ihmir SmithMarsette,WR,MIN,243.00
331,Anthony Miller,WR,CHI,245.14
332,Pooka Williams,RB,CIN,245.45
333,Dwayne Haskins,QB,PIT,260.77


# Bringing It All Together

In [52]:
df['keeper_round'] = np.where(df['keeper_round'] < 1, np.nan, df['keeper_round'])
df[df['round']==5]

,player_id,full_name,fantasy_positions,years_exp,free_agent,trade,waiver,round,pick_no,is_keeper,draft_id,season,draft_owner,draft_owner_id,draft_roster_id,league_id,current_owner,current_owner_id,current_roster_id,held_player_flag,free_agent_flag,traded_player_flag,last_draft_year,last_draft_owner,years_kept_same_owner,keeper_round
26,6790,DAndre Swift,RB,1.0,NaN,NaN,NaN,5.0,42.0,None,591137395731107840,2020.0,tyso0042,3.409441e+17,2.0,722951233224155136,tyso0042,340944113622892544,2,1,0,0,NaN,NaN,NaN,4.0
38,4199,Aaron Jones,RB,4.0,NaN,NaN,NaN,5.0,49.0,True,591137395731107840,2020.0,FillJoo,3.413478e+17,8.0,722951233224155136,FillJoo,341347820613173248,8,1,0,0,2020.0,FillJoo,2.0,2.0
64,4066,Evan Engram,TE,4.0,NaN,NaN,NaN,5.0,47.0,None,591137395731107840,2020.0,MAPavoloni,3.409240e+17,1.0,722951233224155136,MAPavoloni,340924015470219264,1,1,0,0,NaN,NaN,NaN,4.0
72,4988,Nick Chubb,RB,3.0,NaN,NaN,NaN,5.0,50.0,True,591137395731107840,2020.0,magicpajcic,3.411054e+17,7.0,722951233224155136,magicpajcic,341105417952202752,7,1,0,0,2020.0,magicpajcic,2.0,2.0
77,2505,Darren Waller,TE,6.0,NaN,NaN,NaN,5.0,41.0,None,591137395731107840,2020.0,9johnb,3.412669e+17,5.0,722951233224155136,9johnb,341266941933056000,5,1,0,0,NaN,NaN,NaN,4.0
89,6806,JK Dobbins,RB,1.0,NaN,10.0,7.0,5.0,43.0,None,591137395731107840,2020.0,brisbois19,3.409464e+17,3.0,722951233224155136,tyso0042,340944113622892544,2,0,0,1,NaN,NaN,NaN,5.0
92,3969,Leonard Fournette,RB,4.0,NaN,NaN,NaN,5.0,44.0,None,591137395731107840,2020.0,kiwirado,3.412701e+17,6.0,722951233224155136,kiwirado,341270051682283520,6,1,0,0,NaN,NaN,NaN,4.0
101,515,Rob Gronkowski,TE,11.0,NaN,NaN,NaN,5.0,46.0,None,591137395731107840,2020.0,Conorrich,3.410153e+17,4.0,722951233224155136,Conorrich,341015344539787264,4,1,0,0,NaN,NaN,NaN,4.0
125,5892,David Montgomery,RB,2.0,NaN,NaN,NaN,5.0,45.0,None,591137395731107840,2020.0,Rioman,3.417056e+17,10.0,722951233224155136,Rioman,341705565078040576,10,1,0,0,NaN,NaN,NaN,4.0
134,1992,Allen Robinson,WR,7.0,NaN,NaN,NaN,5.0,48.0,True,591137395731107840,2020.0,tweet011,3.413494e+17,9.0,722951233224155136,tweet011,341349429489446912,9,1,0,0,2020.0,tweet011,1.0,3.0


In [53]:
keepcols = ['full_name','fantasy_positions','years_exp','round','pick_no','current_owner','keeper_round']
all_df = df[keepcols]
all_df.head()

,full_name,fantasy_positions,years_exp,round,pick_no,current_owner,keeper_round
0,Michael Thomas,WR,5.0,6.0,51.0,Conorrich,6.0
1,Calvin Ridley,WR,3.0,3.0,21.0,9johnb,2.0
2,Corey Davis,WR,4.0,NaN,NaN,Rioman,12.0
3,Zach Ertz,TE,8.0,3.0,26.0,Conorrich,2.0
4,Cam Newton,QB,10.0,14.0,131.0,MAPavoloni,12.0


In [54]:
all_df[all_df['full_name'].str.contains('Mark') == True]

,full_name,fantasy_positions,years_exp,round,pick_no,current_owner,keeper_round
78,Mark Andrews,TE,3.0,3.0,27.0,tyso0042,2.0


In [55]:
all_df.columns = ['Player_Name','POS','years_exp','round','pick_no','current_owner','keeper_round']

# Cleansing out Player Names
all_df['Player_Name'] = all_df['Player_Name'].str.replace('[^a-zA-Z ]', '')
all_df['Player_Name']

#all_df.head()
all_df[all_df['Player_Name'].str.contains('Mark') == True]

C:\Users\Patrick\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Player_Name,POS,years_exp,round,pick_no,current_owner,keeper_round
78,Mark Andrews,TE,3.0,3.0,27.0,tyso0042,2.0


### Pulling in Draft Order

In [56]:
order = pd.read_csv("C:/Users/Patrick/{}_{}_League_Draft_Order.csv".format(season, commissioner))
order

,current_owner,Draft_Position,Odd_Pick_Order,Even_Pick_Order
0,tyso0042,1,1,10
1,brisbois19,2,2,9
2,Rioman,3,3,8
3,FillJoo,4,4,7
4,MAPavoloni,5,5,6
5,kiwirado,6,6,5
6,9johnb,7,7,4
7,Conorrich,8,8,3
8,tweet011,9,9,2
9,magicpajcic,10,10,1


In [57]:
all_df = pd.merge(all_df, order, how = 'left', on = 'current_owner')
all_df.head()

,Player_Name,POS,years_exp,round,pick_no,current_owner,keeper_round,Draft_Position,Odd_Pick_Order,Even_Pick_Order
0,Michael Thomas,WR,5.0,6.0,51.0,Conorrich,6.0,8,8,3
1,Calvin Ridley,WR,3.0,3.0,21.0,9johnb,2.0,7,7,4
2,Corey Davis,WR,4.0,NaN,NaN,Rioman,12.0,3,3,8
3,Zach Ertz,TE,8.0,3.0,26.0,Conorrich,2.0,8,8,3
4,Cam Newton,QB,10.0,14.0,131.0,MAPavoloni,12.0,5,5,6


### Getting Traded Picks

# api-endpoint
trade_URL = "https://api.sleeper.app/v1/league/{}/traded_picks".format(league_id)

# sending get request and saving the response as response object 
trade_r = requests.get(url = trade_URL) 

# extracting data in json format 
trade_data = trade_r.json() 

all_trade_df = pd.DataFrame.from_dict(trade_data, orient="columns")
all_trade_df.head()


In [58]:
## Saving a copy of df
#all_df_copy = all_df
#all_df = all_df_copy

In [59]:
all_df[all_df['round']==2]

,Player_Name,POS,years_exp,round,pick_no,current_owner,keeper_round,Draft_Position,Odd_Pick_Order,Even_Pick_Order
33,Travis Kelce,TE,8.0,2.0,13.0,tweet011,NaN,9,9,2
36,Amari Cooper,WR,6.0,2.0,17.0,kiwirado,1.0,6,6,5
41,Chris Godwin,WR,4.0,2.0,12.0,FillJoo,1.0,4,4,7
50,Jonathan Taylor,RB,1.0,2.0,18.0,kiwirado,12.0,6,6,5
52,Lamar Jackson,QB,3.0,2.0,14.0,MAPavoloni,1.0,5,5,6
61,Todd Gurley,RB,6.0,2.0,16.0,Rioman,1.0,3,3,8
82,DeAndre Hopkins,WR,8.0,2.0,11.0,magicpajcic,1.0,10,10,1
83,Adam Thielen,WR,8.0,2.0,15.0,Conorrich,1.0,8,8,3
111,DJ Moore,WR,3.0,2.0,20.0,9johnb,1.0,7,7,4
132,LeVeon Bell,RB,8.0,2.0,19.0,brisbois19,12.0,2,2,9


In [60]:
# Removing Suffixes and Titles from player name
all_df['Player_Name'] = all_df.Player_Name.str.replace(r'(\bJr|Sr|III|II|I|V|IV|VI|VII|VIII|IX|X)$', '', regex=True).str.strip()
all_df[all_df['round']==3]

,Player_Name,POS,years_exp,round,pick_no,current_owner,keeper_round,Draft_Position,Odd_Pick_Order,Even_Pick_Order
1,Calvin Ridley,WR,3.0,3.0,21.0,9johnb,2.0,7,7,4
3,Zach Ertz,TE,8.0,3.0,26.0,Conorrich,2.0,8,8,3
28,Kenyan Drake,RB,5.0,3.0,29.0,FillJoo,1.0,4,4,7
31,Tyler Lockett,WR,6.0,3.0,25.0,Rioman,2.0,3,3,8
70,AJ Brown,WR,2.0,3.0,22.0,tyso0042,2.0,1,1,10
78,Mark Andrews,TE,3.0,3.0,27.0,tyso0042,2.0,1,1,10
90,DK Metcalf,WR,2.0,3.0,30.0,magicpajcic,3.0,10,10,1
100,Cooper Kupp,WR,4.0,3.0,23.0,kiwirado,2.0,6,6,5
104,Melvin Gordon,RB,6.0,3.0,28.0,tweet011,2.0,9,9,2


# Merging with Fantasy Football Calculator

### Checking unmerged records to see if valuable players left off

In [61]:
fb_calc[(~fb_calc.Player_Name.isin(all_df.Player_Name))&(~fb_calc.Player_Name.isin(all_df.Player_Name))]

,Player_Name,FBCalc_adp
11,Saquon Barkley,11.0
13,Najee Harris,13.2
44,Kyle Pitts,43.5
58,Javonte Williams,56.7
62,Dak Prescott,60.0
...,...,...
214,Carlos Hyde,174.8
215,Zach Wilson,175.2
216,TreQuan Smith,175.2
217,Carson Wentz,175.9


In [62]:
all_df[(~all_df.Player_Name.isin(fb_calc.Player_Name))&(~all_df.Player_Name.isin(fb_calc.Player_Name))]

,Player_Name,POS,years_exp,round,pick_no,current_owner,keeper_round,Draft_Position,Odd_Pick_Order,Even_Pick_Order
4,Cam Newton,QB,10.0,14.0,131.0,MAPavoloni,12.0,5,5,6
9,Jamison Crowder,WR,6.0,11.0,106.0,Conorrich,10.0,8,8,3
16,Eric Ebron,TE,7.0,NaN,NaN,magicpajcic,12.0,10,10,1
20,Teddy Bridgewater,QB,7.0,NaN,NaN,9johnb,12.0,7,7,4
27,Sammy Watkins,WR,7.0,15.0,149.0,FillJoo,14.0,4,4,7
29,Derek Carr,QB,7.0,NaN,NaN,Conorrich,12.0,8,8,3
30,Keke Coutee,WR,3.0,NaN,NaN,kiwirado,12.0,6,6,5
45,Anthony McFarland,RB,1.0,14.0,140.0,kiwirado,12.0,6,6,5
46,Jalen Reagor,WR,1.0,15.0,142.0,tyso0042,14.0,1,1,10
47,Cam Akers,RB,1.0,4.0,34.0,brisbois19,3.0,2,2,9


In [63]:
all_df[all_df['Player_Name'].str.contains('Dev') == True]
#fb_calc[fb_calc['Player_Name'].str.contains('Scar') == True]

,Player_Name,POS,years_exp,round,pick_no,current_owner,keeper_round,Draft_Position,Odd_Pick_Order,Even_Pick_Order
140,Devin Singletary,RB,2.0,11.0,102.0,tyso0042,9.0,1,1,10


In [64]:
all_df = pd.merge(all_df, fb_calc, how = 'left', on = 'Player_Name')

# Merging with Fantasy Pros ADP

### Checking unmerged records to see if valuable players left off

In [65]:
fp_adp[(~fp_adp.Player_Name.isin(all_df.Player_Name))&(~fp_adp.Player_Name.isin(all_df.Player_Name))]

,Player_Name,Team,POS,FP_ESPN,FP_RTSports,FP_MFL,FP_Fantrax,FP_FFC,FP_Sleeper
7,Saquon Barkley,NYG,RB6,5.0,10.0,11.0,10.0,12.0,6.0
14,Najee Harris,PIT,RB11,15.0,16.0,16.0,15.0,13.0,18.0
45,Kyle Pitts,ATL,TE4,50.0,48.0,48.0,53.0,41.0,40.0
56,Dak Prescott,DAL,QB7,54.0,61.0,55.0,58.0,67.0,58.0
59,Javonte Williams,DEN,RB24,64.0,52.0,67.0,56.0,54.0,62.0
...,...,...,...,...,...,...,...,...,...
465,Cornell Powell,KC,WR161,NaN,NaN,NaN,439.0,NaN,NaN
466,Gardner Minshew,PHI,QB57,NaN,NaN,NaN,440.0,NaN,NaN
467,Jacob Hollister,JAC,TE57,NaN,NaN,NaN,441.0,NaN,NaN
468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
all_df[(~all_df.Player_Name.isin(fp_adp.Player_Name))&(~all_df.Player_Name.isin(fp_adp.Player_Name))]

,Player_Name,POS,years_exp,round,pick_no,current_owner,keeper_round,Draft_Position,Odd_Pick_Order,Even_Pick_Order,FBCalc_adp
47,Cam Akers,RB,1.0,4.0,34.0,brisbois19,3.0,2,2,9,NaN
74,Jeffery Wilson,RB,3.0,NaN,NaN,9johnb,12.0,7,7,4,NaN
89,JK Dobbins,RB,1.0,5.0,43.0,tyso0042,5.0,1,1,10,NaN
108,Lynn Bowden,WR,1.0,NaN,NaN,tyso0042,12.0,1,1,10,NaN
139,Duke Johnson,RB,6.0,14.0,133.0,Conorrich,12.0,8,8,3,NaN


In [67]:
# Merging sleeper data with Fantasy Pros ADP data
all_df = pd.merge(all_df, fp_adp, how = 'left', on = 'Player_Name')
all_df[ ~(all_df['FP_ESPN'].isna() & all_df['FP_RTSports'].isna()) ]

,Player_Name,POS_x,years_exp,round,pick_no,current_owner,keeper_round,Draft_Position,Odd_Pick_Order,Even_Pick_Order,FBCalc_adp,Team,POS_y,FP_ESPN,FP_RTSports,FP_MFL,FP_Fantrax,FP_FFC,FP_Sleeper
0,Michael Thomas,WR,5.0,6.0,51.0,Conorrich,6.0,8,8,3,67.0,NO,WR31,82.0,91.0,73.0,75.0,73.0,63.0
1,Calvin Ridley,WR,3.0,3.0,21.0,9johnb,2.0,7,7,4,18.7,ATL,WR6,20.0,18.0,19.0,18.0,18.0,20.0
2,Corey Davis,WR,4.0,NaN,NaN,Rioman,12.0,3,3,8,98.1,NYJ,WR41,100.0,84.0,93.0,104.0,93.0,110.0
3,Zach Ertz,TE,8.0,3.0,26.0,Conorrich,2.0,8,8,3,156.7,PHI,TE17,170.0,172.0,147.0,171.0,180.0,162.0
4,Cam Newton,QB,10.0,14.0,131.0,MAPavoloni,12.0,5,5,6,NaN,QB32,NaN,NaN,248.0,NaN,247.0,225.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,Michael Pittman,WR,1.0,14.0,139.0,brisbois19,12.0,2,2,9,96.6,IND,WR44,133.0,96.0,115.0,111.0,106.0,113.0
140,Devin Singletary,RB,2.0,11.0,102.0,tyso0042,9.0,1,1,10,90.4,BUF,RB39,137.0,104.0,105.0,109.0,99.0,119.0
141,Justin Herbert,QB,1.0,NaN,NaN,tweet011,12.0,9,9,2,62.1,LAC,QB8,60.0,71.0,60.0,69.0,72.0,56.0
142,Chris Carson,RB,4.0,1.0,10.0,magicpajcic,NaN,10,10,1,24.3,SEA,RB16,39.0,32.0,29.0,33.0,29.0,34.0


# Merging Fantasy Pros Dynasty

In [68]:
# Merging sleeper data with Fantasy Pros Dynasty data
all_df = pd.merge(all_df, fp_dyn, how = 'left', on = 'Player_Name')

In [69]:
all_df[ ( all_df['FP_Dyn_Avg'].isna()) ]

,Player_Name,POS_x,years_exp,round,pick_no,current_owner,keeper_round,Draft_Position,Odd_Pick_Order,Even_Pick_Order,FBCalc_adp,Team,POS_y,FP_ESPN,FP_RTSports,FP_MFL,FP_Fantrax,FP_FFC,FP_Sleeper,FP_Dyn_Avg
74,Jeffery Wilson,RB,3.0,NaN,NaN,9johnb,12.0,7,7,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Merging with Rotoviz Dynasty Data

In [70]:
# Merging sleeper data with Fantasy Pros Dynasty data
all_df = pd.merge(all_df, rv_dyn, how = 'left', on = 'Player_Name')

In [71]:
all_df[ ( all_df['RV_Dyn_Avg'].isna()) ]
#all_df

,Player_Name,POS_x,years_exp,round,pick_no,current_owner,keeper_round,Draft_Position,Odd_Pick_Order,Even_Pick_Order,FBCalc_adp,Team_x,POS_y,FP_ESPN,FP_RTSports,FP_MFL,FP_Fantrax,FP_FFC,FP_Sleeper,FP_Dyn_Avg,Pos,Team_y,RV_Dyn_Avg
65,Ito Smith,RB,3.0,NaN,NaN,Rioman,12.0,3,3,8,NaN,RB108,NaN,NaN,336.0,NaN,NaN,336.0,NaN,333.9,NaN,NaN,NaN
73,Adrian Peterson,RB,14.0,16.0,157.0,kiwirado,15.0,6,6,5,NaN,RB98,NaN,NaN,395.0,NaN,NaN,338.5,NaN,365.0,NaN,NaN,NaN
108,Lynn Bowden,WR,1.0,NaN,NaN,tyso0042,12.0,1,1,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,266.8,NaN,NaN,NaN
139,Duke Johnson,RB,6.0,14.0,133.0,Conorrich,12.0,8,8,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287.3,NaN,NaN,NaN


In [72]:
# Removing Defenses
all_df = all_df[~ all_df['Player_Name'].isna()]
all_df

,Player_Name,POS_x,years_exp,round,pick_no,current_owner,keeper_round,Draft_Position,Odd_Pick_Order,Even_Pick_Order,FBCalc_adp,Team_x,POS_y,FP_ESPN,FP_RTSports,FP_MFL,FP_Fantrax,FP_FFC,FP_Sleeper,FP_Dyn_Avg,Pos,Team_y,RV_Dyn_Avg
0,Michael Thomas,WR,5.0,6.0,51.0,Conorrich,6.0,8,8,3,67.0,NO,WR31,82.0,91.0,73.0,75.0,73.0,63.0,43.1,WR,NOS,47.56
1,Calvin Ridley,WR,3.0,3.0,21.0,9johnb,2.0,7,7,4,18.7,ATL,WR6,20.0,18.0,19.0,18.0,18.0,20.0,14.5,WR,ATL,22.97
2,Corey Davis,WR,4.0,NaN,NaN,Rioman,12.0,3,3,8,98.1,NYJ,WR41,100.0,84.0,93.0,104.0,93.0,110.0,104.4,WR,NYJ,116.98
3,Zach Ertz,TE,8.0,3.0,26.0,Conorrich,2.0,8,8,3,156.7,PHI,TE17,170.0,172.0,147.0,171.0,180.0,162.0,175.6,TE,PHI,161.16
4,Cam Newton,QB,10.0,14.0,131.0,MAPavoloni,12.0,5,5,6,NaN,QB32,NaN,NaN,248.0,NaN,247.0,225.0,NaN,273.9,QB,NEP,194.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Duke Johnson,RB,6.0,14.0,133.0,Conorrich,12.0,8,8,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287.3,NaN,NaN,NaN
140,Devin Singletary,RB,2.0,11.0,102.0,tyso0042,9.0,1,1,10,90.4,BUF,RB39,137.0,104.0,105.0,109.0,99.0,119.0,123.5,RB,BUF,125.80
141,Justin Herbert,QB,1.0,NaN,NaN,tweet011,12.0,9,9,2,62.1,LAC,QB8,60.0,71.0,60.0,69.0,72.0,56.0,67.0,QB,LAC,46.84
142,Chris Carson,RB,4.0,1.0,10.0,magicpajcic,NaN,10,10,1,24.3,SEA,RB16,39.0,32.0,29.0,33.0,29.0,34.0,57.5,RB,SEA,50.19


In [73]:
all_df.columns

Index(['Player_Name', 'POS_x', 'years_exp', 'round', 'pick_no',
       'current_owner', 'keeper_round', 'Draft_Position', 'Odd_Pick_Order',
       'Even_Pick_Order', 'FBCalc_adp', 'Team_x', 'POS_y', 'FP_ESPN',
       'FP_RTSports', 'FP_MFL', 'FP_Fantrax', 'FP_FFC', 'FP_Sleeper',
       'FP_Dyn_Avg', 'Pos', 'Team_y', 'RV_Dyn_Avg'],
      dtype='object')

In [74]:
drop_cols = [ 'POS_y', 'Team_y']

keepers = all_df.drop(columns = drop_cols)

In [75]:
# Getting overall averages for ADP and Dynasty
keepers['Overall_ADP'] = keepers[['FP_ESPN','FP_RTSports','FP_Fantrax','FBCalc_adp']].mean(axis=1)
keepers['Overall_ADP_StDev'] = keepers[['FP_ESPN','FP_RTSports','FP_Fantrax','FBCalc_adp']].std(axis=1)

keepers['Overall_Dynasty_ADP'] = keepers[['FP_Dyn_Avg'
                                          ,'RV_Dyn_Avg'
                                         ]].mean(axis=1)
keepers['Overall_Dynasty_ADP_StDev'] = keepers[['FP_Dyn_Avg'
                                                ,'RV_Dyn_Avg'
                                               ]].std(axis=1)

In [76]:
# Only keeping overall columns
drop_cols = ['FP_ESPN',  'FP_RTSports', 'FP_Fantrax','FBCalc_adp', 'FP_Dyn_Avg']
keepers = keepers.drop(columns = drop_cols)

In [77]:
# Dropping players not eligible for keepers
#keepers = keepers[~keepers['keeper_round'].isna()]
keepers

,Player_Name,POS_x,years_exp,round,pick_no,current_owner,keeper_round,Draft_Position,Odd_Pick_Order,Even_Pick_Order,Team_x,FP_MFL,FP_FFC,FP_Sleeper,Pos,RV_Dyn_Avg,Overall_ADP,Overall_ADP_StDev,Overall_Dynasty_ADP,Overall_Dynasty_ADP_StDev
0,Michael Thomas,WR,5.0,6.0,51.0,Conorrich,6.0,8,8,3,NO,73.0,73.0,63.0,WR,47.56,78.750,10.210289,45.330,3.153696
1,Calvin Ridley,WR,3.0,3.0,21.0,9johnb,2.0,7,7,4,ATL,19.0,18.0,20.0,WR,22.97,18.675,0.942956,18.735,5.989194
2,Corey Davis,WR,4.0,NaN,NaN,Rioman,12.0,3,3,8,NYJ,93.0,93.0,110.0,WR,116.98,96.525,8.704549,110.690,8.895403
3,Zach Ertz,TE,8.0,3.0,26.0,Conorrich,2.0,8,8,3,PHI,147.0,180.0,162.0,TE,161.16,167.425,7.196469,168.380,10.210622
4,Cam Newton,QB,10.0,14.0,131.0,MAPavoloni,12.0,5,5,6,QB32,NaN,225.0,NaN,QB,194.84,247.500,0.707107,234.370,55.903862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Duke Johnson,RB,6.0,14.0,133.0,Conorrich,12.0,8,8,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287.300,NaN
140,Devin Singletary,RB,2.0,11.0,102.0,tyso0042,9.0,1,1,10,BUF,105.0,99.0,119.0,RB,125.80,110.100,19.579922,124.650,1.626346
141,Justin Herbert,QB,1.0,NaN,NaN,tweet011,12.0,9,9,2,LAC,60.0,72.0,56.0,QB,46.84,65.525,5.301179,56.920,14.255273
142,Chris Carson,RB,4.0,1.0,10.0,magicpajcic,NaN,10,10,1,SEA,29.0,29.0,34.0,RB,50.19,32.075,6.035106,53.845,5.168951


In [78]:
# Identifying the value of the pick used for keeper
keepers['keep_pick_num'] = np.where(keepers['keeper_round'] % 2 == 0, 
             ((num_teams *keepers['keeper_round']) + keepers['Even_Pick_Order'])   , 
             ((num_teams *keepers['keeper_round']) + keepers['Odd_Pick_Order']) )
keepers

,Player_Name,POS_x,years_exp,round,pick_no,current_owner,keeper_round,Draft_Position,Odd_Pick_Order,Even_Pick_Order,Team_x,FP_MFL,FP_FFC,FP_Sleeper,Pos,RV_Dyn_Avg,Overall_ADP,Overall_ADP_StDev,Overall_Dynasty_ADP,Overall_Dynasty_ADP_StDev,keep_pick_num
0,Michael Thomas,WR,5.0,6.0,51.0,Conorrich,6.0,8,8,3,NO,73.0,73.0,63.0,WR,47.56,78.750,10.210289,45.330,3.153696,63.0
1,Calvin Ridley,WR,3.0,3.0,21.0,9johnb,2.0,7,7,4,ATL,19.0,18.0,20.0,WR,22.97,18.675,0.942956,18.735,5.989194,24.0
2,Corey Davis,WR,4.0,NaN,NaN,Rioman,12.0,3,3,8,NYJ,93.0,93.0,110.0,WR,116.98,96.525,8.704549,110.690,8.895403,128.0
3,Zach Ertz,TE,8.0,3.0,26.0,Conorrich,2.0,8,8,3,PHI,147.0,180.0,162.0,TE,161.16,167.425,7.196469,168.380,10.210622,23.0
4,Cam Newton,QB,10.0,14.0,131.0,MAPavoloni,12.0,5,5,6,QB32,NaN,225.0,NaN,QB,194.84,247.500,0.707107,234.370,55.903862,126.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Duke Johnson,RB,6.0,14.0,133.0,Conorrich,12.0,8,8,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287.300,NaN,123.0
140,Devin Singletary,RB,2.0,11.0,102.0,tyso0042,9.0,1,1,10,BUF,105.0,99.0,119.0,RB,125.80,110.100,19.579922,124.650,1.626346,91.0
141,Justin Herbert,QB,1.0,NaN,NaN,tweet011,12.0,9,9,2,LAC,60.0,72.0,56.0,QB,46.84,65.525,5.301179,56.920,14.255273,122.0
142,Chris Carson,RB,4.0,1.0,10.0,magicpajcic,NaN,10,10,1,SEA,29.0,29.0,34.0,RB,50.19,32.075,6.035106,53.845,5.168951,NaN


In [79]:
# 50% ADP and 50% Dynasty
keepers['Overall_Rank'] = ( keepers[['Overall_ADP','Overall_Dynasty_ADP']].sum(axis=1)) / 2

In [80]:
# Getting Overall Value : Rank - Pick Number
keepers['Overall_Value'] = keepers['keep_pick_num'] - keepers['Overall_Rank']
keepers.sort_values( by = 'Overall_Value', ascending=False)

,Player_Name,POS_x,years_exp,round,pick_no,current_owner,keeper_round,Draft_Position,Odd_Pick_Order,Even_Pick_Order,Team_x,FP_MFL,FP_FFC,FP_Sleeper,Pos,RV_Dyn_Avg,Overall_ADP,Overall_ADP_StDev,Overall_Dynasty_ADP,Overall_Dynasty_ADP_StDev,keep_pick_num,Overall_Rank,Overall_Value
50,Jonathan Taylor,RB,1.0,2.0,18.0,kiwirado,12.0,6,6,5,IND,13.0,15.0,9.0,RB,5.54,8.975,2.910183,4.770,1.088944,125.0,6.8725,118.1275
126,Justin Jefferson,WR,1.0,12.0,111.0,tweet011,12.0,9,9,2,MIN,24.0,23.0,24.0,WR,13.75,23.000,1.414214,12.275,2.085965,122.0,17.6375,104.3625
86,Terry McLaurin,WR,2.0,12.0,118.0,MAPavoloni,11.0,5,5,6,WAS,31.0,31.0,30.0,WR,35.26,28.900,1.762574,29.730,7.820601,115.0,29.3150,85.6850
137,TJ Hockenson,TE,2.0,15.0,146.0,Conorrich,14.0,8,8,3,DET,62.0,58.0,55.0,TE,54.75,58.150,3.399510,57.275,3.570889,143.0,57.7125,85.2875
99,Tee Higgins,WR,1.0,NaN,NaN,brisbois19,12.0,2,2,9,CIN,65.0,64.0,67.0,WR,52.23,61.600,3.006659,45.515,9.496444,129.0,53.5575,75.4425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Miles Sanders,RB,2.0,1.0,8.0,tweet011,NaN,9,9,2,PHI,35.0,39.0,38.0,RB,40.61,37.450,2.100000,43.505,4.094148,NaN,40.4775,NaN
98,Christian McCaffrey,RB,4.0,1.0,1.0,9johnb,NaN,7,7,4,CAR,1.0,1.0,1.0,RB,1.78,1.075,0.150000,1.440,0.480833,NaN,1.2575,NaN
123,Dalvin Cook,RB,4.0,1.0,4.0,kiwirado,NaN,6,6,5,MIN,2.0,2.0,2.0,RB,4.25,2.125,0.250000,4.225,0.035355,NaN,3.1750,NaN
135,Clyde EdwardsHelaire,RB,1.0,1.0,5.0,Rioman,NaN,3,3,8,KC,23.0,21.0,25.0,RB,24.80,22.375,3.092329,25.600,1.131371,NaN,23.9875,NaN
